In [ ]:
import pandas as pd, numpy as np

pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings("ignore")

import datetime

In [ ]:
links = pd.read_csv('interchange_segments selected.csv')
links['dir'] = links['ROUTE_ID'].map(lambda x : x.split('-')[3])
links = links.loc[(links.dir == '000') | (links.dir == '010')]
print(len(links))
links.head()

In [ ]:
#horizontal curvature 
links['DegHCurve_LWA'] = ((links['CU_A_LENGTH']*1.7)+(links['CU_B_LENGTH']*4.45)+
                          (links['CU_C_LENGTH']*6.95)+(links['CU_D_LENGTH']*11.2)+
                          (links['CU_E_LENGTH']*20.95)+(links['CU_F_LENGTH']*30))/(links['SECTION_LENGTH'])

In [ ]:
#vertical grade
links['DegGrade_LWA'] = ((links['GR_A_LENGTH']*0.2)+(links['GR_B_LENGTH']*1.45)+
                          (links['GR_C_LENGTH']*3.45)+(links['GR_D_LENGTH']*5.45)+
                          (links['GR_E_LENGTH']*7.45)+(links['GR_F_LENGTH']*9.45))/(links['SECTION_LENGTH'])

In [ ]:
#contains just cardinal direction
linksc = links.loc[links.AllRds_Dir == 'Cardinal'].reset_index(drop=True)
linksc.head()

In [ ]:
ln = linksc.sort_values(by=['ROUTE_ID', 'BEGIN_POINT', 'END_POINT'], ascending = True).reset_index(drop=True)
ln['LNID'] = ln.index

In [ ]:
print(len(ln))
ln.tail()

In [ ]:
revid = 1
i = 0
while i < len(ln)-1:
    ln.loc[i, 'LNSegID'] = revid
    if ln.loc[i, 'ROUTE_ID'] == ln.loc[i+1, 'ROUTE_ID']:
        if (ln.loc[i, 'THROUGH_LANES'] != ln.loc[i+1, 'THROUGH_LANES']) |\
           (ln.loc[i, 'RURAL_URBAN'] != ln.loc[i+1, 'RURAL_URBAN']) |\
           (ln.loc[i, 'DegHCurve_LWA'] != ln.loc[i+1, 'DegHCurve_LWA']) |\
           (ln.loc[i, 'DegGrade_LWA'] != ln.loc[i+1, 'DegGrade_LWA']) |\
           (ln.loc[i, 'AADT'] != ln.loc[i+1, 'AADT']) |\
           (ln.loc[i, 'LANE_WIDTH'] != ln.loc[i+1, 'LANE_WIDTH']) |\
           (ln.loc[i, 'SPEED_LIMIT_LWA'] != ln.loc[i+1, 'SPEED_LIMIT_LWA']) |\
           (ln.loc[i, 'SHOULDER_WIDTH_L'] != ln.loc[i+1, 'SHOULDER_WIDTH_L']) |\
           (ln.loc[i, 'SHOULDER_WIDTH_R'] != ln.loc[i+1, 'SHOULDER_WIDTH_R']) |\
           (ln.loc[i, 'TYPE_TERRAIN'] != ln.loc[i+1, 'TYPE_TERRAIN']) |\
           (ln.loc[i, 'TRUCK_PERC'] != ln.loc[i+1, 'TRUCK_PERC']):
            revid += 1
    else:
        revid += 1
    i += 1
ln.loc[i, 'LNSegID'] = revid

In [ ]:
ln.tail()

In [ ]:
first = lambda x: x.iloc[0]
lwa = lambda x: np.average(x, weights=ln.loc[x.index, "SECTION_LENGTH"])

ln_grouped = ln.groupby('LNSegID', as_index=False).agg({'ROUTE_ID':first,
                                          'BEGIN_POINT': 'min',
                                          'END_POINT': 'max',
                                          'RURAL_URBAN':first,
                                          'DegHCurve_LWA':lwa,
                                          'DegGrade_LWA':lwa,
                                          'AADT': lwa,
                                          'TRUCK_PERC': lwa,
                                          'TYPE_TERRAIN':lwa,
                                          'THROUGH_LANES': lwa,
                                          'LANE_WIDTH': lwa,
                                           'SHOULDER_WIDTH_L':lwa,
                                           'SHOULDER_WIDTH_R':lwa,
                                           'SPEED_LIMIT_LWA':lwa,
                    
                                          
                                          })

In [ ]:
ln_grouped["SECTION_LENGTH"] =  abs(ln_grouped['END_POINT'] - ln_grouped['BEGIN_POINT'])
ln_grouped.head()

In [ ]:
print(f"Total Homogenous segments both directions: {len(ln_grouped)}")

In [ ]:
#choose cardinal direction
# use cardinal direction MPs and length to aggregate data for both sides
# Pick minimum speed both directions, also average speed both directions
#aggregate speed data by date-hour from ln links to LNSegID level.


In [ ]:
#links change rt unique 010 to 000

links['RTMod'] = links['ROUTE_ID'].str[:-3]+'000'
links['BMP'] = links['BEGIN_POINT']
links['EMP'] = links['END_POINT']
links['LinkLength'] = abs(links['EMP'] - links['BMP'])
links.head()

In [ ]:
ln_grouped.to_csv('homogenous_segments.csv', index=False)

In [ ]:
here_links = links[['RTMod', 'BMP', 'EMP', 'LinkLength', 'AllRds_Dir', 'LinkDir']]
here_links.head()

In [ ]:
ln_grouped.head(3)

In [ ]:
#Generate links in each LNSegID (homogenous segment) for both directions
segs_here = pd.DataFrame()
for i in range(len(ln_grouped)):  #range(len(ln_grouped))
    rt = ln_grouped.loc[i, 'ROUTE_ID']
    sbmp = ln_grouped.loc[i, 'BEGIN_POINT']
    semp = ln_grouped.loc[i, 'END_POINT']
    segid = ln_grouped.loc[i, 'LNSegID']
    slen = ln_grouped.loc[i, 'SECTION_LENGTH']
    
    within_links = here_links.loc[(here_links.RTMod == rt) & (here_links.BMP < semp) &
                                 (here_links.EMP > sbmp)].sort_values('BMP').reset_index(drop=True)
    
    within_links.loc[within_links.loc[within_links['AllRds_Dir'] == 'Cardinal','BMP'].idxmin(),'BMP'] = sbmp
    within_links.loc[within_links.loc[within_links['AllRds_Dir'] == 'Cardinal','EMP'].idxmax(),'EMP'] = semp
    within_links.loc[within_links.loc[within_links['AllRds_Dir'] != 'Cardinal','BMP'].idxmin(),'BMP'] = sbmp
    within_links.loc[within_links.loc[within_links['AllRds_Dir'] != 'Cardinal','EMP'].idxmax(),'EMP'] = semp
    
    within_links['LNSegID'] = segid
    within_links['Segment_Length'] = slen
    
    segs_here = segs_here.append(within_links, ignore_index=True)

In [ ]:
segs_here.tail()

In [ ]:
segs_here.to_csv('homogenous_segments_links.csv', index=False)

### Speed OverLay

In [ ]:
ln_grouped = pd.read_csv('homogenous_segments.csv')
seglinks = pd.read_csv('homogenous_segments_links.csv')

In [ ]:
ln_grouped.head()

In [ ]:
ln_grouped[ln_grouped.LANE_WIDTH==11]

In [ ]:
seglinks.head(10)

In [ ]:
spd = pd.read_csv('2018_second_half_speed.csv')
spd = spd.sort_values(['LinkDir', 'DateTime'])
spd.head(10)

In [ ]:
#check if Datetime has minute
spd['minute'] = spd['DateTime'].str[-2:]

In [ ]:
spd = spd[['LinkDir', 'DateTime', 'Hour', 'Mean','dow']]
spd.head()

In [ ]:
spd['DateTime'] = spd['DateTime'].astype('datetime64[ns]')

In [ ]:
spd.info()

In [ ]:
#merge speeds onto links
link_speed = pd.merge(seglinks, spd, on='LinkDir', how='inner')

In [ ]:
link_speed.info(show_counts=True)

In [ ]:
link_speed['MeanTT'] = link_speed['LinkLength']/link_speed['Mean']

In [ ]:
link_speed.head()

In [ ]:
link_tt = link_speed.groupby(['LNSegID', 'AllRds_Dir', 'DateTime'], as_index=False).agg({'LinkLength':'sum',
                                                                                         'MeanTT':'sum',
                                                                               })
link_tt.head()

In [ ]:
link_tt['AvgSpeed'] = round(link_tt['LinkLength']/link_tt['MeanTT'],3)
link_tt.head()

In [ ]:
linkc = link_tt.loc[link_tt.AllRds_Dir == 'Cardinal', ['LNSegID', 'DateTime', 'AvgSpeed']]
linkc.columns = ['LNSegID', 'DateTime', 'AvgSpeedC']
linkc.head()

In [ ]:
linkc.info(show_counts=True)

In [ ]:
linknc = link_tt.loc[link_tt.AllRds_Dir == 'NonCardinal', ['LNSegID', 'DateTime', 'AvgSpeed']]
linknc.columns = ['LNSegID', 'DateTime', 'AvgSpeedNC']
linknc.head()

In [ ]:
linknc.info(show_counts=True)

In [ ]:
segspeeds = pd.merge(linkc, linknc, on=['LNSegID', 'DateTime'], how='left')
segspeeds.info(show_counts=True)

In [ ]:
segspeeds['AvgSpeedbd'] = segspeeds[['AvgSpeedC', 'AvgSpeedNC']].mean(axis=1)
segspeeds['AvgSpeedbd'] = round(segspeeds['AvgSpeedbd'],2)

In [ ]:
segspeeds['MinSpeedbd'] = segspeeds[['AvgSpeedC', 'AvgSpeedNC']].min(axis=1)
segspeeds['MinSpeedbd'] = round(segspeeds['MinSpeedbd'],2)

In [ ]:
segspeeds.info()

In [ ]:
segspeeds.head()

In [ ]:
segspeeds.to_csv('Homogenous_Segment_speeds.csv', index=False)

### Weather

In [ ]:
segspeeds = pd.read_csv('Homogenous_Segment_speeds.csv')

In [ ]:
segspeeds.info(show_counts=True)

In [ ]:
weather = pd.read_csv('Weather/weather_data_second_half_2018.csv')
weather.head()

In [ ]:
weather_stations = weather.drop_duplicates('mesonetNetSiteName', ignore_index=True)
print(len(weather_stations))
weather_stations = weather_stations[['mesonetNetSiteName','mesonetGeoLat','mesonetGeoLong']]
weather_stations.head()

In [ ]:
#weather_stations.to_csv('weather_stations.csv', index=False)

In [ ]:
weather_relevant = weather[['mesonetNetSiteName','kytcDateExistsET','mesonetDewpoint', 'mesonetAirTemperature', 
                            'mesonetPrecipitationToday','mesonetWindSpeed']]

weather_relevant['kytcDateExistsET'] = weather_relevant['kytcDateExistsET'].str[:-5] + '00'

weather_relevant.columns = ['weatherstation','DateTime', 'dewpoint', 'airtemp', 'precipitation','windspeed']
weather_relevant.head()

In [ ]:
weather_relevant['DateTime'] = weather_relevant['DateTime'].astype('datetime64[ns]')
weather_relevant.info()

In [ ]:
weather_relevant = weather_relevant.groupby(['weatherstation', 'DateTime'], as_index=False).agg({'dewpoint':'mean',
                                                                                               'airtemp':'mean',
                                                                                            'precipitation':'sum',
                                                                                             'windspeed':'mean'})

In [ ]:
weather_relevant.to_csv('homogenous_segments_weather_data.csv', index=False)

In [ ]:
weather_relevant.info()

In [ ]:
hsegs = pd.read_csv('homogenous_segments_with_weather_stations.csv')
hsegs.head()

### Waze Vehicle On Shoulder

In [ ]:
vos = pd.read_csv('Waze_alerts_no_duplicates_clustered.csv')
vos['Year'] = 2018
vos.head()

In [ ]:
def convertdatetime(year,month,day,hour):
    return format(datetime.datetime(year, month, day, hour), '%Y-%m-%d %H:%M')

In [ ]:
vos['DateTime'] = vos.apply(lambda x: convertdatetime(x['Year'], x['month'], x['day'], x['hour']), axis=1)
vos = vos[['kytcrtuniqueinitial', 'kytcbmpinitial', 'kytcempinitial', 'DateTime', 'wazealertincidentsubtype']]
vos.head()

In [ ]:
vos['ROUTE_ID'] = vos['kytcrtuniqueinitial'].str[:-3] + '000'
vos.head()

In [ ]:
vos_segs = pd.DataFrame()
for i in range(len(ln_grouped)):
    rt = ln_grouped.loc[i, 'ROUTE_ID'].replace(' ', '')
    bmp = ln_grouped.loc[i, 'BEGIN_POINT']
    emp = ln_grouped.loc[i, 'END_POINT']
    segid = ln_grouped.loc[i, 'LNSegID']
    
    voslinks = vos.loc[(vos.ROUTE_ID == rt) & (vos.kytcbmpinitial<emp) & (vos.kytcempinitial>bmp)] 
    voslinks['LNSegID'] = segid
    
    vos_segs = vos_segs.append(voslinks, ignore_index=True)

In [ ]:
vos_segs.head()

In [ ]:
vos_segs = vos_segs[['LNSegID','DateTime','wazealertincidentsubtype']]
vos_segs.head()

In [ ]:
vos_segs['DateTime'] = vos_segs['DateTime'].astype('datetime64[ns]')

In [ ]:
vos_segs.info()

In [ ]:
#final
vos_segs_grouped = vos_segs.groupby(['LNSegID','DateTime'], as_index=False).agg({'wazealertincidentsubtype':'count'})
vos_segs_grouped.columns = ['LNSegID','DateTime', 'voshcount']
vos_segs_grouped.head()

In [ ]:
vos_segs_grouped.info()

## All together

In [ ]:
hsegs.rename(columns = {'mesonetNetSiteName':'weatherstation'}, inplace=True)
hsegs.head()

In [ ]:
hsegs.info()

In [ ]:
segspeeds.info()

In [ ]:
segs_spd = pd.merge(hsegs, segspeeds, on='LNSegID', how='left', copy=False)
segs_spd.head()

In [ ]:
segs_spd.info(show_counts=True)

In [ ]:
weather_relevant.info(show_counts=True)

In [ ]:
segs_spd.to_csv('Homogenous_Segment_speeds.csv', index=False)

In [ ]:
segs_spd_weather = pd.merge(segs_spd, weather_relevant, on=['weatherstation','DateTime'], how='inner', copy=False)
segs_spd_weather.head()

In [ ]:
segs_spd_weather.info(show_counts=True)

In [ ]:
segs_speed_weather_vos = pd.merge(segs_spd_weather, vos_segs_grouped, on=['LNSegID','DateTime'], how='left')
segs_speed_weather_vos.head()

In [ ]:
segs_speed_weather_vos.info(show_counts=True)

In [ ]:
segs_speed_weather_vos['voshcount'] = segs_speed_weather_vos['voshcount'].fillna(0)

In [ ]:
segs_speed_weather_vos.info(show_counts=True)

In [ ]:
segs_speed_weather_vos.head()

In [ ]:
segs_speed_weather_vos.to_csv('homogenous_segments_spdweathervosh.csv')

In [ ]:
#speed
#weather
#vosh
#volume

In [ ]:
segs_speed_weather_vos.drop(['ROUTE_ID', 'BEGIN_POINT', 'END_POINT','weatherstation'], axis=1, inplace=True)

In [ ]:
segs_speed_weather_vos['dt'] = pd.to_datetime(segs_speed_weather_vos['DateTime'])

In [ ]:
segs_speed_weather_vos['hour'] = segs_speed_weather_vos['dt'].dt.hour

In [ ]:
segs_speed_weather_vos['dow'] = segs_speed_weather_vos['dt'].dt.dayofweek

In [ ]:
segs_speed_weather_vos['weekend'] = segs_speed_weather_vos['dow'].map(lambda x: 1 if x>4 else 0)

In [ ]:
segs_speed_weather_vos.head()

In [ ]:
segs_speed_weather_vos['weekend'].min()

In [ ]:
segs_speed_weather_vos.to_parquet('segs_speed_weather_vos.parquet', index=False)

### volume

In [ ]:
segs_speed_weather_vos = pd.read_parquet('segs_speed_weather_vos.parquet')
segs_speed_weather_vos.head()

In [ ]:
segs_speed_weather_vos['hour'].unique()

In [ ]:
factors = pd.read_csv('ATR_HourlyFactors_2014_2015_2016.csv')
factors = factors[factors.fun_c == 1]
factors.head()

In [ ]:
factors = factors[['Hour','dayofweek','perc']]
factors.columns = ['hour','weekend','perc']
factors.head()

In [ ]:
segs_speed_weather_vos_vol = pd.merge(segs_speed_weather_vos, factors, on=['hour', 'weekend'], how='left')

In [ ]:
segs_speed_weather_vos_vol['hvol'] = (segs_speed_weather_vos_vol['AADT']/2) * segs_speed_weather_vos_vol['perc']

In [ ]:
segs_speed_weather_vos_vol.drop(['dt','hour','dow','perc'], axis=1, inplace=True)

In [ ]:
segs_speed_weather_vos_vol.head()

In [ ]:
segs_speed_weather_vos_vol.to_parquet('segs_speed_weather_vos_vol.parquet', index=False)

### crashes

In [ ]:
segs_speed_weather_vos_vol = pd.read_parquet('segs_speed_weather_vos_vol.parquet')

In [ ]:
crashes = pd.read_csv('interstate_crashes.csv')
crashes.head()

In [ ]:
crashes['DateTime'] = crashes['timestamp'].str[:-5] + '00'
crashes['RTmod'] = crashes['ROUTE_ID'].str[:-3] + '000'

In [ ]:
hsegs['rt'] = hsegs['ROUTE_ID'].str.replace(' ','')

In [ ]:
crashes.head()

In [ ]:
crashes['LNSegID'] = None
for i in range(len(crashes)): #
    
    rt = crashes.loc[i, 'RTmod']
    mp = crashes.loc[i, 'MilePoint']
    
    seg = hsegs.loc[(hsegs.rt == rt) & (hsegs.BEGIN_POINT <= mp)& (hsegs.END_POINT >= mp) ].sort_values('BEGIN_POINT')
    
    if len(seg) > 0:
        segid = seg.LNSegID.values[0]
        crashes.loc[i, 'LNSegID'] = segid

In [ ]:
crashes.head()

In [ ]:
crashes = crashes.loc[pd.notnull(crashes.LNSegID)]
crashes.head()

In [ ]:
crashes = crashes[['LNSegID','DateTime','MasterFile']]

In [ ]:
crashes_count = crashes.groupby(['LNSegID', 'DateTime'], as_index=False).agg({'MasterFile':'count'})
crashes_count.columns = ['LNSegID', 'DateTime', 'crashcount']

In [ ]:
crashes_count['DateTime'] = crashes_count['DateTime'].astype('datetime64[ns]')

In [ ]:
crashes_count.head()

In [ ]:
crashes_count.info(show_counts=True)

In [ ]:
segs_speed_weather_vos_vol.info(show_counts=True)

In [ ]:
data = pd.merge(segs_speed_weather_vos_vol, crashes_count, on=['LNSegID','DateTime'], how='left')

In [ ]:
data['crashcount'] = data['crashcount'].fillna(0)

In [ ]:
data.info(show_counts=True)

In [ ]:
data.to_parquet('all_data.parquet', index=False)

In [ ]:
data['RURAL'] = data['RURAL_URBAN'].map(lambda x: 1 if x=='Rural' else 0)
data['NOT_FLAT_TERRAIN'] = data['TYPE_TERRAIN'].map(lambda x: 1 if (x != 1) else 0)

data['VehShoulderPresent'] = data['voshcount'].map(lambda x: 1 if x>=1 else 0)
data['Crash'] = data['crashcount'].map(lambda x: 1 if x>=1 else 0)

In [ ]:
data['AvgSpdminusSL'] = data['AvgSpeedbd'] - data['SPEED_LIMIT_LWA']
data['MinSpdminusSL'] = data['MinSpeedbd'] - data['SPEED_LIMIT_LWA']

In [ ]:
data.to_parquet('data_allcols.parquet', index=False)

### Data

In [ ]:
data = pd.read_parquet('data_allcols.parquet')
data.head()

In [ ]:
data.info(show_counts=True)

In [ ]:
data_sub = data[['LNSegID','RURAL','DegHCurve_LWA', 'DegGrade_LWA', 'TRUCK_PERC', 'THROUGH_LANES', 'LANE_WIDTH',
              'SHOULDER_WIDTH_L', 'SHOULDER_WIDTH_R', 'SPEED_LIMIT_LWA', 'SECTION_LENGTH','DateTime', 'AvgSpeedbd',
              'MinSpeedbd', 'dewpoint', 'airtemp', 'precipitation', 'windspeed','weekend', 'hvol','NOT_FLAT_TERRAIN', 
             'AvgSpdminusSL', 'MinSpdminusSL', 'VehShoulderPresent', 'crashcount','Crash']]

In [ ]:
data.columns

In [ ]:
data_sub.info(show_counts=True)

In [ ]:
data_sub.to_parquet('data_formodel_final.parquet', index=False)

In [ ]:
data_sub.to_csv('model_data.csv', index=False)

### Assess Correlation

In [ ]:
#data = pd.read_csv('model_data.csv')
data = data_sub.copy()
data.head()

In [ ]:
#filter out less than 5mph speed seg-datetime
data = data[data.AvgSpeedbd >= 5]

In [ ]:
data['DateTime'] = pd.to_datetime(data['DateTime'])
data.head()

In [ ]:
data.DateTime.min(), data.DateTime.max()

In [ ]:
data.sort_values(['LNSegID','DateTime'], ascending=True, inplace=True)
data.reset_index(drop=True, inplace=True)
data['TimeStep'] = data['DateTime'].factorize()[0] + 1
data['ID'] = data['LNSegID'].factorize()[0] + 1
data.head()

In [ ]:
summary = data.groupby(['LNSegID'], as_index=False).agg({'Crash':'sum', 'VehShoulderPresent':'sum','TimeStep':'max'})
summary

In [ ]:
cols = [ 'ID','TimeStep','RURAL', 'DegHCurve_LWA', 'DegGrade_LWA', 'TRUCK_PERC','THROUGH_LANES', 'LANE_WIDTH', 
        'SHOULDER_WIDTH_L', 'SHOULDER_WIDTH_R', 'SPEED_LIMIT_LWA', 'SECTION_LENGTH','AvgSpeedbd','MinSpeedbd',
        'dewpoint', 'airtemp', 'precipitation', 'windspeed','weekend', 'hvol', 'NOT_FLAT_TERRAIN', 'AvgSpdminusSL',
        'MinSpdminusSL','VehShoulderPresent','crashcount','Crash']

In [ ]:
data_sub = data[cols]
data_sub.head()

In [ ]:
from math import log
def ln(x):
    return log(x)

data_sub['Ln_hvol'] = data_sub['hvol'].map(ln)
data_sub['Ln_seglen'] = data_sub['SECTION_LENGTH'].map(ln)
data_sub.head()

In [ ]:
data_sub['ratio_spdlim_avgspd'] = data_sub['SPEED_LIMIT_LWA']/data_sub['AvgSpeedbd']
data_sub['ratio_spdlim_avgspd_capped'] = np.where(data_sub['ratio_spdlim_avgspd'] < 1, 1, data_sub['ratio_spdlim_avgspd'])
#cap at speed limit
data_sub['AvgSpdminusSL_capped'] = np.where(data_sub['AvgSpdminusSL'] > 0, 0, data_sub['AvgSpdminusSL'])

data_sub.head()

In [ ]:
data_sub.describe().transpose()

In [ ]:
data_sub.to_csv('E:/WAZE/data_integrated/model_data_segs_all.csv', index=False)

### with crash

In [ ]:
has_crash = summary.query("Crash > 0")
has_crash_segs = has_crash.LNSegID.tolist()
data_has_crash = data[data.LNSegID.isin(has_crash_segs)]
data_has_crash.sort_values(['LNSegID','DateTime'], ascending=True, inplace=True)
data_has_crash['TimeStep'] = data_has_crash['DateTime'].factorize()[0] + 1
data_has_crash.reset_index(drop=True, inplace=True)

In [ ]:
data_has_crash['ID'] = data_has_crash['LNSegID'].factorize()[0] + 1

In [ ]:
cols = [ 'ID','TimeStep','RURAL', 'DegHCurve_LWA', 'DegGrade_LWA', 'TRUCK_PERC','THROUGH_LANES', 'LANE_WIDTH', 
        'SHOULDER_WIDTH_L', 'SHOULDER_WIDTH_R', 'SPEED_LIMIT_LWA', 'SECTION_LENGTH','AvgSpeedbd','MinSpeedbd',
        'dewpoint', 'airtemp', 'precipitation', 'windspeed','weekend', 'hvol', 'NOT_FLAT_TERRAIN', 'AvgSpdminusSL',
        'MinSpdminusSL','VehShoulderPresent','crashcount','Crash']

In [ ]:
data_has_crash_sub = data_has_crash[cols]
data_has_crash_sub.head()

In [ ]:
check = data_has_crash_sub.groupby(['ID'], as_index=False).agg({'Crash':'sum', 'VehShoulderPresent':'sum','TimeStep':'max'})
check.head()

In [ ]:
check.TimeStep.max()

In [ ]:
data_has_crash_sub.hvol.max(), data_has_crash_sub.SECTION_LENGTH.max(),

In [ ]:
from math import log
def ln(x):
    return log(x)

data_has_crash_sub['Ln_hvol'] = data_has_crash_sub['hvol'].map(ln)
data_has_crash_sub['Ln_seglen'] = data_has_crash_sub['SECTION_LENGTH'].map(ln)
data_has_crash_sub.head()

In [ ]:
data_has_crash_sub['ratio_spdlim_avgspd'] = data_has_crash_sub['SPEED_LIMIT_LWA']/data_has_crash_sub['AvgSpeedbd']
data_has_crash_sub['ratio_spdlim_avgspd_capped'] = np.where(data_has_crash_sub['ratio_spdlim_avgspd'] < 1, 1, data_has_crash_sub['ratio_spdlim_avgspd'])
#cap at speed limit
data_has_crash_sub['AvgSpdminusSL_capped'] = np.where(data_has_crash_sub['AvgSpdminusSL'] > 0, 0, data_has_crash_sub['AvgSpdminusSL'])

data_has_crash_sub.head()

In [ ]:
data_has_crash_sub.describe().transpose()

In [ ]:
data_has_crash_sub.to_csv('E:/WAZE/data_integrated/model_data_segs_with_crashes.csv', index=False)

### data_has_crash_sub

In [ ]:
data_has_crash_sub = pd.read_csv('New Folder/model_data_segs_with_crashes.csv')
data_has_crash_sub.head()

In [ ]:
data_has_crash_sub.LANE_WIDTH.unique()

In [ ]:
data_has_crash_sub.loc[data_has_crash_sub.LANE_WIDTH == 12, 'SHOULDER_WIDTH_R'].unique()

In [ ]:
correlations = data_has_crash_sub.corr()

In [ ]:
correlations_all = data_sub.corr()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np


def CorrMtx(df, dropDuplicates = True):

    # Your dataset is already a correlation matrix.
    # If you have a dateset where you need to include the calculation
    # of a correlation matrix, just uncomment the line below:
    # df = df.corr()

    # Exclude duplicate correlations by masking uper right values
    if dropDuplicates:    
        mask = np.zeros_like(df, dtype=np.bool_)
        mask[np.triu_indices_from(mask)] = True

    # Set background color / chart style
    sns.set_style(style = 'white')

    # Set up  matplotlib figure
    f, ax = plt.subplots(figsize=(11, 9))

    # Add diverging colormap from red to blue
    cmap = sns.diverging_palette(250, 10, as_cmap=True)

    # Draw correlation plot with or without duplicates
    if dropDuplicates:
        sns.heatmap(df, mask=mask, cmap=cmap, 
                square=True,
                linewidth=.5, cbar_kws={"shrink": .5}, ax=ax)
    else:
        sns.heatmap(df, cmap=cmap, 
                square=True,
                linewidth=.5, cbar_kws={"shrink": .5}, ax=ax)

In [ ]:
CorrMtx(correlations, dropDuplicates = True)

In [ ]:
correlations.to_csv('E:/WAZE/data_integrated/data_has_crash_correlations_mar2.csv')
correlations_all.to_csv('E:/WAZE/data_integrated/data_all_correlations_mar2.csv')

In [ ]:
wz = pd.read_csv('Z:/Waze/Segmentation/data_reduced.csv')
wz.head()

In [ ]:
cor = wz.corr()

In [ ]:
CorrMtx(cor)

In [ ]:
cor[cor]